# **Sumário** 📘


1.   **Resumo**
2.   **Introdução**
3.   **Objetivos**
4.   **Metodologia**
4.   **Modelo**
4.   **Usando o Modelo**
5.   **Otimizando o Modelo**
5.   **Visualisando parametros otimizados**
5.   **Resultados**
6.   **Referências**



## **Ferramenta de Benchmarking para Transistores de Película Fina** ⚡

## **Aluno**:

Rodrigo Santos Batista – rsb6@cin.ufpe.br

CIn ­ UFPE

## **Projeto de Pesquisa**:

Avaliação comparativa de tecnologias emergentes para computação próxima e

dentro de sensores (BEeTle)

## **Coordenador**:

Stefan Michael Blawid – sblawid@cin.ufpe.br

CIn ­ UFPE


### **Resumo** 📩

> Transistores de película fina são chaves eletrônicas com uma corrente fluindo do terminal fonte para o terminal dreno controlada pela tensão aplicada ao terceiro terminal, a porta. Os modelos de benchmark precisam reproduzir razoavelmente bem as características de corrente-tensão (IV) dos TFTs com um número mínimo de parâmetros. Aqui, razoavelmente significa que os parâmetros do modelo podem ser extraídos de forma confiável e sem ambiguidade das curvas IV experimentais.





## **Usando o Modelo** 💻


### **Carregando dados da amostra** 📚


---



In [1]:
# Load the used classes
from modules_otft._imports import *
from modules_otft._read_data import ReadData 
from modules_otft._model import TFTModel 
from modules_otft._grafics import  TFTGraphicsPlot
from modules_otft._menu import TFTMenu 

from modules_otft._global_vars import * 
from modules_otft._config import *
from modules_otft._utils import * 


In [2]:
# Instantiating the used classes
plot = TFTGraphicsPlot()
menu = TFTMenu()
read = ReadData()

In [3]:
show_varibles()

| path: datas/Fig1ab/tipo p
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| experimental_data_scale_transfer: A
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| experimental_data_scale_output: uA
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| current_typic: uA
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| loaded_voltages: -2, -50, -30, -40, -50
-------------------------------

In [4]:
# @title ### **Load parameters local / remote directory.**
# @markdown In this section, you can load the intrinsic parameters of the model you want to optimize. The load can be done locally, by choosing your `json` file containing all the values ​​to be loaded, parameters `Vtho, delta, n, l, lam, Vgcrit, Jth, Rs`

# @markdown Note, also, that it is necessary to provide the respective values ​​of `resistance_scale`, `current_carry` and `with_transistor`, which are respectively: resistance values, current values ​​- where the transistor typically operates - and transistor width (associated technology). If you want to add a value other than the default, add it to the field. Remember that the values ​​in the `JTH` and `Rs` fields will be multiples of the values ​​defined in these fields and the value of `with_transistor` is in units of `[cm]`. So make sure you provide consistent values ​​so that the template output can also be consistent

# @markdown ---


ld_voltages = load_voltages(loaded_voltages)

type_curve_plot = get_type_plot(type_curve_plot)

shift_list = calculate_shift_list(shift_volt_data, ld_voltages, curves_transfer)

list_tension_shift = get_shift_list(read, curves_transfer, shift_list, ld_voltages)

path_voltages = read.read_files_experimental(path, list_tension_shift)

Vv, Id, input_voltage, n_points, count_transfer, count_output = read.load_data(type_read_data_exp, path_voltages, current_typic, experimental_data_scale_transfer, experimental_data_scale_output, type_curve_plot)


print("Shift value    : " + f'{shift_list}')
print()

print("List volt shift: " +  f'{list_tension_shift}')
print()
list_curves = menu.view_path_reads(path_voltages, list_tension_shift)

print()
print('The table above shows all the files that were read in the directory, if you want to filter files so that only those that are of interest are read, do this by passing numbers that correspond to the order of the experiment curves,')
print('separated by ,. For example: 0:tranfer-5v , 3:output-40v and so on')
print('\n\n')

path_voltages, list_tension_shift, list_tension = filter_and_load_files(read, select_files, list_curves, list_tension_shift, ld_voltages, path)

print()

print("List volt shift  : " f'{list_tension_shift}' )
print("List tension     : " f'{list_tension}' )
print()
_ = menu.view_path_reads(path_voltages, list_tension_shift)
print('\n\n')


tp_tst = get_transistor_type(type_of_transistor)
resistance = get_resistance(resistance_scale)
current = get_current(current_carry)
width_t = get_transistor_width(with_transistor)
load_parameters = load_coefficients(loaded_parameters)


print()
menu.print_values(load_parameters)
print()

mode_idleak, load_idleak = load_idleak_parameters(idleak_mode, loaded_idleak)

Shift value    : [0, 0, 0]

List volt shift: [-2, -50, -30, -40, -50]

--------------------------------------------
--------------------------------------------
|                READ FILES                |
--------------------------------------------
--------------------------------------------
|            transfer-2V.csv            |
--------------------------------------------
|             transfer-50V.csv             |
--------------------------------------------
|              output-30V.csv              |
--------------------------------------------
|              output-40V.csv              |
--------------------------------------------
|              output-50V.csv              |
--------------------------------------------
--------------------------------------------

The table above shows all the files that were read in the directory, if you want to filter files so that only those that are of interest are read, do this by passing numbers that correspond to the order of the e

In [12]:
# @title ##### ***GRAFICS PLOT OPTION:*** { run: "auto" }
option = 'show both curves' # @param ["Show Transfer curve", "Show output curve", "show both curves"]

plot = initialize_graphics_plot(image_size_width, image_size_height)
model = instance_model(read, TFTModel, n_points, type_curve_plot, load_parameters, input_voltage, Vv, load_idleak, width_t, count_transfer, tp_tst, experimental_data_scale_transfer, current_typic, resistance, current)
in_model_data, in_exp_data, out_model_data, out_exp_data = load_experimental_data(read, count_transfer, Vv, Id, model, count_output)
plot_curves(option, plot, list_tension, list_tension_shift, count_transfer, in_model_data, out_model_data, in_exp_data, out_exp_data, shift_list, select_files, current_typic, type_curve_plot)
read.compute_relative_distance(Id, model)








**************************************************************************************************************************************************

|RELATIVE ERROR BETWEEN CURVES: 147082113134.7026

**************************************************************************************************************************************************


### **Model Optimization** ⚡

This optimization section focuses on estimating the optimal parameters for a MOSFET transistor given a sequence of input voltage $V$ [V] and current $I$ [mA], using the mathematical model introduced in section [5]. The goal here is to estimate these optimal parameters using the `curve_fit` function, as also introduced in section [4] (for more details, refer to that section). The parameters in question are listed below:

- $V_{tho}$: Gate threshold voltage of the transistor.
- $\Delta$: Channel modulation coefficient.
- $N$: Number of transistor channels.
- $L$: Channel length of the transistor.
- $\Lambda$: Channel modulation length.
- $V_{crit}$: Transistor breakdown voltage.
- $J_{th}$: Transistor saturation current.
- $R_s$: Transistor series resistance.



In [6]:
# @title #### **OPTIMIZING THE MODEL.** { run: "auto" }
lw_bounds, up_bounds = get_bounds(lower_bounds, upper_bounds)
tlr_factor = get_tolerance_factor(tolerance_factor)
lw_bounds, up_bounds = configure_bounds(default_bounds, lw_bounds, up_bounds)

model_id = create_model_opt(TFTModel, input_voltage, n_points, type_curve_plot, current_typic, 
                            experimental_data_scale_transfer, load_idleak, mode_idleak, count_transfer, 
                            resistance, current)

optimizer = create_optimizer(current_typic, experimental_data_scale_transfer, experimental_data_scale_output,
                             type_read_data_exp, path_voltages, type_curve_plot, optimization_method, (lw_bounds, up_bounds))

configure_optimizer(optimizer, default_bounds, tlr_factor)

coeff_opt, coeff_error, text_verbose = optimize_model(optimizer, model_id, load_parameters, *path_voltages)

# Exibição dos resultados
print('\n\n\n')
print('*' * 200)
print(text_verbose.capitalize())
print('*' * 200)
print('\n\n\n')


----------------------------------------------------------------------------------------------------
TRUST REGION REFLECTIVE (TRF) MODE
----------------------------------------------------------------------------------------------------





********************************************************************************************************************************************************************************************************
`gtol` termination condition is satisfied.
function evaluations 89, initial cost 2.3642e+05, final cost 5.0757e-01, first-order optimality 3.94e-07.

********************************************************************************************************************************************************************************************************






In [7]:
# @title #### **SHOW MODEL PARAMETERS OPTIMIZED.** { run: "auto" }
# @markdown ### **Loading Parameters and Optimizing the Model** ⚡
# @markdown > This section is dedicated to visualizing the parameters obtained through optimization. To do this, simply choose any of the options below:
# @markdown ##### ***Show the list coefficients optimized:***
# @markdown ##### **--> Vtho, delta, n, l, lam, Vgcrit, Jth, Rs**
# @markdown ---

option = 'Show the table of values opt' # @param ["Show the coeff values","Show the initial values", "Show the table of values opt", "coeff opt", "coeff error"]
show_model_parameters_optimized(menu, option, load_parameters, coeff_opt, coeff_error, current_typic, resistance)



---------------------------
 OPTIMIZED COEFFICIENTS:

╒══════════════════════╤═════════════════╤═══════════════════╤═════════════════════════════════════════╕
│ Parameter            │   Initial Value │   Optimized Value │   Associated Error (standard deviation) │
╞══════════════════════╪═════════════════╪═══════════════════╪═════════════════════════════════════════╡
│ VTHO [V]             │          8.57   │       12          │                              9.75017    │
├──────────────────────┼─────────────────┼───────────────────┼─────────────────────────────────────────┤
│ DELTA                │          0.0123 │        0.00714077 │                              0.0322358  │
├──────────────────────┼─────────────────┼───────────────────┼─────────────────────────────────────────┤
│ N                    │        108      │      120.986      │                             51.9999     │
├──────────────────────┼─────────────────┼───────────────────┼─────────────────────────────────────────┤


### **Comparing** ✅

---
This section aims to evaluate the model's performance relative to the initial data. Visual observation allows assessing whether the model significantly outperformed the initial coefficients.

In this section, simply execute the cells below; no explicit modifications are required.

---

In [14]:
# @title #### **Print grafics plots of model Optimizad and model data.** { run: "auto" }
# @markdown ### ***Choice the plot that you want to view:***
# @markdown ---
option = 'show both curves comp' # @param ["Show Transfer curve opt", "Show output curve opt", "show both curves opt", "Show Transfer curve comp", "Show output curve comp", "show both curves comp"]

model_opt = create_optimized_model(read, TFTModel, n_points, type_curve_plot, coeff_opt, input_voltage, Vv, load_idleak, width_t, count_transfer, tp_tst, experimental_data_scale_transfer, current_typic, resistance, current)

in_model_data_opt, in_exp_data_opt, out_model_data_opt, out_exp_data_opt, in_model_data_comp, in_exp_data_comp, out_model_data_comp, out_exp_data_comp = get_model_data(read, count_transfer, count_output, Vv, Id, model_opt, model, compare=True)

plot_curves(option, plot, list_tension, list_tension_shift, count_transfer, in_model_data_comp, out_model_data_comp, in_exp_data_comp, out_exp_data_comp, shift_list, select_files, current_typic, type_curve_plot, compare=True)

read.compute_relative_distance(Id, model_opt)
print('\n\n\n\n\n')








**************************************************************************************************************************************************

|RELATIVE ERROR BETWEEN CURVES: 9738880877.4632

**************************************************************************************************************************************************






